In [1]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 2. SZÖVEGELEMZÉS (ÉRZELEM OSZTÁLYOZÁS, KATEGÓRIA PREDIKÁLÁS)

In [3]:
connection = sqlite3.connect("C:/Users/Zen/Desktop/Egyetem/Szakdolgozat/Data/podcasts.db")

In [4]:
# 2.1 ÉRZELEM OSZTÁLYOZÁS

# Adat betöltés
sentiment_data = pd.read_sql("""
    select
        content,
        case when rating < 3 then 0
            when rating > 4 then 2
            else 1
        end as sentiment
    from reviews
""", connection)

In [5]:
# 2.2 KATEGÓRIA PREDIKCIÓ - TELJES
podcast_reviews = pd.read_sql("select reviews.content, categories.category from reviews, categories where reviews.podcast_id = categories.podcast_id", connection)
podcast_reviews_content = podcast_reviews['content'].values.tolist()
podcast_reviews_category = podcast_reviews['category'].values.tolist()

In [6]:
#podcast_reviews['category_id'] = podcast_reviews['category'].factorize()[0]

In [7]:
from sklearn.naive_bayes import MultinomialNB # Naive Bayes
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(podcast_reviews_content, podcast_reviews_category)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [ ]:
def predict_category(s):
    pred = model.predict([s])
    return pred

In [ ]:
print(predict_category('bank system, valuta, credit card'))

In [8]:
import pickle
pickle.dump(model, open('fullcatpred.pkl','wb'))

In [13]:
# 2.3 KATEGÓRIA PREDIKCIÓ - RÉSZLEGES
categories = ['technology', 'arts-food','hinduism', 'spirituality']
categories

['technology', 'arts-food', 'hinduism', 'spirituality']

In [14]:
def decision_maker(subject, categories):
    x = False
    for cat in categories:
        if subject == cat:
            x = True  
    return x

In [15]:
podcast_content = []
podcast_category = []

for ind in podcast_reviews.index:
    if (decision_maker(podcast_reviews['category'][ind], categories)):
        podcast_content.append(podcast_reviews['content'][ind])
        podcast_category.append(podcast_reviews['category'][ind])

64898

In [95]:
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(podcast_content, podcast_category)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [109]:
print(predict_category(''))

['spirituality']
